Importing the relevant libraries.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

Loading dataset and setting inital variables

In [2]:
model_name='perceptron'
dataset_name="addition_dataset_2"
validation_split=0.2
total_epoch=50
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

Splitting dataset into training and testing

In [3]:
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=validation_split, random_state=42)

Defining and compiling the model

In [4]:
# Define the model
dset_model_Perceptron = tf.keras.Sequential([
    layers.Dense(1)
])

# Compile the model
dset_model_Perceptron.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define TensorBoard callback
tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')

Training the model

In [5]:
# Start timer for training time
start_time = time.time()

# Train the model with TensorBoard callback
history = dset_model_Perceptron.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

# End timer for training time
training_time = time.time() - start_time

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 986us/step - accuracy: 0.0000e+00 - loss: 678789632.0000 - val_accuracy: 2.5000e-04 - val_loss: 333033824.0000
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - accuracy: 0.0000e+00 - loss: 261317216.0000 - val_accuracy: 2.5000e-04 - val_loss: 107326256.0000
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.0000e+00 - loss: 81310320.0000 - val_accuracy: 2.5000e-04 - val_loss: 24535076.0000
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - accuracy: 0.0000e+00 - loss: 17264364.0000 - val_accuracy: 2.5000e-04 - val_loss: 3480003.2500
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - accuracy: 0.0000e+00 - loss: 2177467.5000 - val_accuracy: 2.5000e-04 - val_loss: 255507.5469
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - accuracy: 0.0000e+00 - loss: 142686.9688 - val_accuracy: 2.5000e-04 - val_loss: 8226.6660
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - accuracy: 0.0000e+00 - loss: 4007.5837

Making prediction and calculating the metrics

In [6]:
# Make predictions
predictions = dset_model_Perceptron.predict(X_test)

# Calculate metrics
# Calculate accuracy with threshold
threshold = 0.5  # Define the threshold
correct_predictions = np.sum(np.abs(y_test - predictions.flatten()) <= threshold)
total_predictions = len(y_test)
accuracy = correct_predictions / total_predictions  # Final accuracy
mse = mean_squared_error(y_test, predictions)  # Mean Squared Error
mae = mean_absolute_error(y_test, predictions)  # Mean Absolute Error
rmse = np.sqrt(mse)  # Root Mean Squared Error
f1 = f1_score(y_test, predictions.round(), average='micro')  # F1 Score

# Calculate inference time
start_time = time.time()
predictions = dset_model_Perceptron.predict(X_test)
inference_time = time.time() - start_time

# Print and store metrics
metrics = {
    'Model type': model_name,
    'dataset': dataset_name,
    'Accuracy': accuracy,
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'F1 Score': f1,
    'Training Time': training_time,
    'Inference Time': inference_time
}

print("Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")

# Store metrics in a list
metrics_list = list(metrics.values())
pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step
Metrics:
Model type: perceptron
dataset: addition_dataset_2
Accuracy: 1.0
MSE: 0.05638053600668246
MAE: 0.23360124123096465
RMSE: 0.23744585910620228
F1 Score: 1.0
Training Time: 22.666709184646606
Inference Time: 0.11066842079162598


Making a function to generate models for different dataset

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy']):
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        neurons, layer_type = layer_config
        if layer_type == 'dense':
            model.add(layers.Dense(neurons))
        elif layer_type == 'dropout':
            model.add(layers.Dropout(neurons))
        elif layer_type == 'activation':
            model.add(layers.Activation(neurons))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')

    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2"):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels):
    threshold = 0.5  # Define the threshold
    correct_predictions = np.sum(np.abs(true_labels - predictions.flatten()) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, predictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, predictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), predictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }

# Load dataset
dataset_name = "addition_dataset_2"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 176566640.0000 - val_accuracy: 0.0000e+00 - val_loss: 40225732.0000
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 0.0000e+00 - loss: 25304700.0000 - val_accuracy: 0.0000e+00 - val_loss: 2668801.2500
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.0000e+00 - loss: 1408125.5000 - val_accuracy: 2.5000e-04 - val_loss: 49373.0977
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - accuracy: 0.0000e+00 - loss: 22088.9355 - val_accuracy: 2.5000e-04 - val_loss: 185.2908
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - accuracy: 0.0000e+00 - loss: 70.7563 - val_accuracy: 2.5000e-04 - val_loss: 0.1200
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.0000e+00 - loss: 0.0775 - val_accuracy: 2.5000e-04 - val_loss: 0.0395
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.0000e+00 - loss: 0.0395 - val_accuracy: 2.5000e-04 - val_loss

In [4]:
# Load dataset
dataset_name = "substraction_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 901us/step - accuracy: 0.0000e+00 - loss: 32999668.0000 - val_accuracy: 2.5000e-04 - val_loss: 200477.4688
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - accuracy: 2.7153e-05 - loss: 61796.8359 - val_accuracy: 2.5000e-04 - val_loss: 2.0011
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 3.3209e-06 - loss: 0.4779 - val_accuracy: 2.5000e-04 - val_loss: 0.0213
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 7.1847e-05 - loss: 0.0220 - val_accuracy: 2.5000e-04 - val_loss: 0.0207
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 2.6210e-04 - loss: 0.0211 - val_accuracy: 2.5000e-04 - val_loss: 0.0205
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 3.8047e-05 - loss: 0.0212 - val_accuracy: 2.5000e-04 - val_loss: 0.0203
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - accuracy: 8.9988e-05 - loss: 0.0205 - val_accuracy: 2.5000e-04 - val_loss: 0.0203
Epoch 8/50
500/500

In [5]:
# Load dataset
dataset_name = "doubling_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 919us/step - accuracy: 0.0000e+00 - loss: 404240352.0000 - val_accuracy: 2.5000e-04 - val_loss: 257967552.0000
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - accuracy: 0.0000e+00 - loss: 227182016.0000 - val_accuracy: 0.0000e+00 - val_loss: 131959088.0000
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.0000e+00 - loss: 111914528.0000 - val_accuracy: 0.0000e+00 - val_loss: 58920136.0000
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.0000e+00 - loss: 48665076.0000 - val_accuracy: 0.0000e+00 - val_loss: 21791256.0000
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - accuracy: 0.0000e+00 - loss: 17112342.0000 - val_accuracy: 0.0000e+00 - val_loss: 6228235.5000
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.0000e+00 - loss: 4552762.5000 - val_accuracy: 0.0000e+00 - val_loss: 1244061.2500
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.0000e+00 - loss: 8

In [6]:
# Load dataset
dataset_name = "simple_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 888us/step - accuracy: 0.0000e+00 - loss: 259024576.0000 - val_accuracy: 2.5000e-04 - val_loss: 146256224.0000
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.0000e+00 - loss: 121991432.0000 - val_accuracy: 2.5000e-04 - val_loss: 60912344.0000
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.0000e+00 - loss: 48876856.0000 - val_accuracy: 2.5000e-04 - val_loss: 20386994.0000
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.0000e+00 - loss: 15556241.0000 - val_accuracy: 2.5000e-04 - val_loss: 5007388.0000
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.0000e+00 - loss: 3517842.7500 - val_accuracy: 2.5000e-04 - val_loss: 805112.8125
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.0000e+00 - loss: 520289.2812 - val_accuracy: 2.5000e-04 - val_loss: 73434.8438
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - accuracy: 0.0000e+00 - loss: 42843.980

In [7]:
# Load dataset
dataset_name = "simple_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 870us/step - accuracy: 0.0000e+00 - loss: 74690936.0000 - val_accuracy: 0.0000e+00 - val_loss: 24646334.0000
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.0000e+00 - loss: 17297280.0000 - val_accuracy: 2.5000e-04 - val_loss: 3641549.0000
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.0000e+00 - loss: 2281712.0000 - val_accuracy: 2.5000e-04 - val_loss: 244342.1875
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - accuracy: 0.0000e+00 - loss: 134401.2969 - val_accuracy: 2.5000e-04 - val_loss: 5829.6509
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.0000e+00 - loss: 2716.5352 - val_accuracy: 2.5000e-04 - val_loss: 37.4851
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - accuracy: 1.1117e-04 - loss: 15.3713 - val_accuracy: 2.5000e-04 - val_loss: 0.7211
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 4.1018e-04 - loss: 0.6972 - val_accuracy: 2.5000e-04 - 